In [1]:
import os
import sys
import pickle
import torch
import yaml
import numpy as np
import matplotlib.pyplot as plt

sys.path.append('../')
from utils import parameter_manager

params = yaml.load(open('../config.yaml', 'r'), Loader = yaml.FullLoader)
pm = parameter_manager.ParameterManager(params=params)

Using MPI version 4.0, 1 processes


In [2]:
no_pillars = pickle.load(open("no_pillars.pkl","rb"))
print(no_pillars.keys())
for key, value in no_pillars.items():
    if key == 'radii':
        #rad = [0 for rad in key]
        no_pillars[key] = [0 for _ in value]

dict_keys(['flux', 'near_fields_1550', 'near_fields_1060', 'near_fields_1300', 'near_fields_1650', 'near_fields_2881', 'eps_data', 'sim_time', 'radii'])


In [3]:
no_pillars['flux']   # i can't use this flux information because i didn't obtain it correctly. i used mp.get_fluxes. should have used 
                     #    res = sim.get_eigenmode_coefficients(flux_object, [1], eig_parity=mp.ODD_Y)
                     #    coeffs = res.alpha
                     #    flux = abs(coeffs[0,0,0]**2)

{'source_flux': 0.3014212577274805, 'downstream_flux': 0.3014220864099054}

In [4]:
# so let's get the dft slice just downstream of the source:
def get_slice(data, pm, value):

    # put value on a (0 to pm.cell_z) scale - meep defines the cell on a (-cell_z/2 to cell_z/2) scale
    #value = pm.fr_center
    value = value + pm.cell_z / 2 

    # length of the cell in microns 
    cell_min = 0  # um
    cell_max = pm.cell_z  # um

    # length of the cell in pixels
    pix_min = 0   
    #pix_max = data['eps_data'].squeeze().shape[2]
    pix_max = 395

    #pix_max = data['near_fields_1550']['ex'].squeeze().shape[2]
    temp = int(((value - cell_min) / (cell_max - cell_min)) * (pix_max - pix_min) + pix_min)

    #pml_pix = (data['eps_data'].squeeze().shape[2] - data['near_fields_1550']['ex'].squeeze().shape[2]) // 2
    pml_pix = (395 - 270) // 2

    return temp - pml_pix

In [5]:
dataset = torch.load("/develop/data/spie_journal_2023/kube_dataset/pp_dataset.pt")
dataset.keys()

dict_keys(['all_near_fields', 'radii', 'phases', 'derivatives', 'sim_times', 'flux_info'])

## Let's get S0_source and S0_downstream:

In [6]:
# first we need to get our respective z slices of the fields

source_z = get_slice(dataset, pm, pm.src_fr_center)
downstream_z = get_slice(dataset, pm, pm.fr_center)

source_plane_x = no_pillars['near_fields_1550']['ex'][:,:,source_z]
source_plane_y = no_pillars['near_fields_1550']['ey'][:,:,source_z]
source_plane_z = no_pillars['near_fields_1550']['ez'][:,:,source_z]

ds_plane_x = no_pillars['near_fields_1550']['ex'][:,:,downstream_z]
ds_plane_y = no_pillars['near_fields_1550']['ey'][:,:,downstream_z]
ds_plane_z = no_pillars['near_fields_1550']['ez'][:,:,downstream_z]

In [7]:
# now let's get E_0

# E_0_src = np.sqrt(np.abs(source_plane_x)**2 + np.abs(source_plane_y)**2 + np.abs(source_plane_z)**2)
# I_src = 0.5 * np.average(E_0_src) **2

In [9]:
#I_src

In [38]:
E_0_ds = np.sqrt(np.abs(ds_plane_x)**2 + np.abs(ds_plane_y)**2 + np.abs(ds_plane_z)**2)
I_ds = 0.5 * E_0_ds **2
avg_I_no_pillar = np.average(I_ds)

In [39]:
avg_I_no_pillar

0.02513237255237675

## Now let's get the intensity from one of the samples with pillars:

In [32]:
dataset['all_near_fields']['near_fields_1550'][1].shape # this is the downstream slice, we don't have the source slice with this pp dataset.
                        # sample, (x, y, z), (amp, phase), (shape) 

torch.Size([1, 3, 2, 166, 166])

In [33]:
sample = dataset['all_near_fields']['near_fields_1550'][1].squeeze()
sample.shape

torch.Size([3, 2, 166, 166])

In [34]:
amp = sample[:,0,:,:] # 3 components
phase = sample[:,1,:,:]

complex_field = amp * torch.exp(1j * phase)
complex_field.shape

torch.Size([3, 166, 166])

In [35]:
E_0_sample = np.sqrt(np.abs(complex_field[0])**2 + np.abs(complex_field[1])**2 + np.abs(complex_field[2])**2)
I_sample = 0.5 * E_0_sample **2
avg_I_sample = np.average(I_sample)

In [36]:
avg_I_sample

0.028457316950934543

In [44]:
for i in range(5):
    sample = dataset['all_near_fields']['near_fields_1550'][i].squeeze()
    amp = sample[:,0,:,:] # 3 components
    phase = sample[:,1,:,:]

    complex_field = amp * torch.exp(1j * phase)
    E_0_sample = np.sqrt(np.abs(complex_field[0])**2 + np.abs(complex_field[1])**2 + np.abs(complex_field[2])**2)
    I_sample = 0.5 * E_0_sample **2
    avg_I_sample = np.average(I_sample)
    print(avg_I_sample *1000)

25.766037013569154
28.457316950934544
25.492425333991108
18.673535301035347
26.340154754787093


In [45]:
avg_I_no_pillar * 1000

25.132372552376747